In [71]:
import numpy as np
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
# Get the uploaded file name
file_name = next(iter(uploaded))

# Load the CSV file using NumPy
data_train = np.genfromtxt(file_name, delimiter=',')

In [ ]:
n1=0
n2=0
n3=0
n4=0
n5=0
for i in range(0,60000):
  if data_train[i][10]==1:
    n1=n1+1
  if data_train[i][10]==2:
    n2=n2+1
  if data_train[i][10]==3:
    n3=n3+1
  if data_train[i][10]==4:
    n4=n4+1
  if data_train[i][10]==5:
    n5=n5+1

In [ ]:
p1=n1/60000
p2=n2/60000
p3=n3/60000
p4=n4/60000
p5=n5/60000

In [ ]:
import math

In [ ]:
def density(mu,sigma,x):
  # e=-.5(((x-mu)/sigma)**2)
  # exp_value=math.exp(e)
  # val=(math.sqrt(2*math.pi))*sigma
  diff=np.zeros((10,1))
  for i in range(0,10):
    diff[i]=x[i]-mu[i]
  e=-0.5*((diff.T)@(np.linalg.inv(sigma))@diff)
  evalue=math.exp(e)
  #value=1/(((2*(math.pi))**5)*math.sqrt(np.linalg.det(sigma)))
  det=math.sqrt(np.linalg.det(sigma))
  value=(2*math.pi)**5
  return evalue/(det*value)


In [ ]:
def gamma1_calculator(xi,mu1,mu2,sigma1,sigma2,lambda1,lambda2):
  val1=lambda1*density(mu1,sigma1,xi)
  val2=lambda2*density(mu2,sigma2,xi)

  return val1/(val1+val2)


def gamma2_calculator(xi,mu1,mu2,sigma1,sigma2,lambda1,lambda2):
  val1=lambda1*density(mu1,sigma1,xi)
  val2=lambda2*density(mu2,sigma2,xi)

  return val2/(val1+val2)





In [ ]:
def mu1_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  mu1=np.zeros((10,1))
  sum1=0
  gamma_i=0
  for i in range(0,size):
    sum1=sum1+gamma1_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    gamma_i=gamma1_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    for j in range(0,10):
      mu1[j]=mu1[j]+gamma_i*data[i][j]
    gamma_i=0

  return mu1/sum1


def mu2_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  mu2=np.zeros((10,1))
  sum2=0
  gamma_i=0
  for i in range(0,size):
    sum2=sum2+gamma2_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    gamma_i=gamma2_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    for j in range(0,10):
      mu2[j]=mu2[j]+gamma_i*data[i][j]
    gamma_i=0

  return mu2/sum2

In [ ]:
def lambda1_calculator(arr,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  sum=0
  for i in range(0,size):
    sum=sum+gamma1_calculator(arr[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)

  return sum/size

def lambda2_calculator(arr,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  sum=0
  for i in range(0,size):
    sum=sum+gamma2_calculator(arr[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)

  return sum/size

In [ ]:
def sgma1_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  sigma1=np.zeros((10,10))
  diff=np.zeros((10,1))
  sum1=0
  for i in range(0,size):
    sum1=sum1+gamma1_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    gamma_i=gamma1_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    for j in range(0,10):
      diff[j]=(data[i][j]-mu1[j])
    sigma1=sigma1+gamma_i*(diff@(diff.T))
    diff=np.zeros((10,1))

  return sigma1/sum1


def sgma2_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  sigma2=np.zeros((10,10))
  diff=np.zeros((10,1))
  sum2=0
  for i in range(0,size):
    sum2=sum2+gamma2_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    gamma_i=gamma2_calculator(data[i],mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    for j in range(0,10):
      diff[j]=(data[i][j]-mu2[j])
    sigma2=sigma2+gamma_i*(diff@(diff.T))
    diff=np.zeros((10,1))

  return sigma2/sum2


In [ ]:
mu1_prev=np.zeros((10,1))
mu2_prev=np.zeros((10,1))
mu1=np.ones((10,1))
mu2=np.ones((10,1))
sigma1_prev= np.identity(10, dtype="float")
sigma2_prev= np.identity(10, dtype="float")
sigma1= 2*np.identity(10, dtype="float")
sigma2= 2*np.identity(10, dtype="float")
lambda1_prev=1
lambda2_prev=1
lambda1=2
lambda2=2

mu1_error=np.linalg.norm((mu1-mu1_prev))
mu2_error=np.linalg.norm((mu2-mu2_prev))
sigma1_error=np.linalg.norm((sigma1-sigma1_prev))
sigma2_error=np.linalg.norm((sigma2-sigma2_prev))
lambda1_error=abs(lambda1-lambda1_prev)
lambda2_error=abs(lambda2-lambda2_prev)




In [ ]:
x1=np.zeros((n1,10))
x2=np.zeros((n2,10))
x3=np.zeros((n3,10))
x4=np.zeros((n4,10))
x5=np.zeros((n5,10))

In [ ]:
k1=0
k2=0
k3=0
k4=0
k5=0


for i in range(0,60000):
    if data_train[i][10]==1:
      for j in range(0,10):
        x1[k1][j]=data_train[i][j]
      k1=k1+1
    elif data_train[i][10]==2:
      for j in range(0,10):
        x2[k2][j]=data_train[i][j]
      k2=k2+1
    elif data_train[i][10]==3:
      for j in range(0,10):
        x3[k3][j]=data_train[i][j]
      k3=k3+1
    elif data_train[i][10]==4:
      for j in range(0,10):
        x4[k4][j]=data_train[i][j]
      k4=k4+1
    elif data_train[i][10]==5:
      for j in range(0,10):
        x5[k5][j]=data_train[i][j]
      k5=k5+1

In [ ]:
def class_cond_dens(xi,mu1,mu2,sigma1,sigma2,lambda1,lambda2):
  value=lambda1*density(mu1,sigma1,xi)+lambda2*density(mu2,sigma2,xi)
  return value

In [ ]:
# prob0=0.5*np.zeros((60000,1))
# prob1=0.6*np.zeros((60000,1))

In [ ]:
# gamma1=np.ones((60000,1))
# gamma2=np.ones((60000,1))

def ccd_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev):
  prob0=0.5*np.zeros((size,1))
  prob1=0.6*np.zeros((size,1))

 # while mu1_error>=0.1 and mu2_error>=0.1 and sigma1_error>=0.1 and sigma2_error >= 0.1 and lambda1_error >= 0.1 and lambda2_error >= 0.1:
  MAE=0.1
  while MAE>=0.005:
    value1=np.zeros((10,1))
    value2=np.zeros((10,1))
    t1=0
    t2=0
    s1=np.zeros((10,10))
    s2=np.zeros((10,10))
    value1=mu1_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    value2=mu2_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    t1=lambda1_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    t2=lambda2_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    s1=sgma1_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    s2=sgma2_calculator(data,size,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
    mu1_prev=mu1
    mu2_prev=mu2
    mu1=value1
    mu2=value2
    sigma1_prev=sigma1
    sigma2_prev=sigma2
    sigma1=s1
    sigma2=s2
    lambda1_prev=lambda1
    lambda2_prev=lambda2
    lambda1=t1
    lambda2=t2
    prob0=prob1
    for i in range(0,size):
      prob1[i]=class_cond_dens(data[i],mu1,mu2,sigma1,sigma2,lambda1,lambda2)

    error=prob1-prob0
    MAE=np.mean(abs(error))



  return mu1,mu2,sigma1,sigma2,lambda1,lambda2




In [ ]:
mu1_cls1,mu2_cls1,sigma1_cls1,sigma2_cls1,lambda1_cls1,lambda2_cls1=ccd_calculator(x1,n1,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
mu1_cls2,mu2_cls2,sigma1_cls2,sigma2_cls2,lambda1_cls2,lambda2_cls2=ccd_calculator(x2,n2,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
mu1_cls3,mu2_cls3,sigma1_cls3,sigma2_cls3,lambda1_cls3,lambda2_cls3=ccd_calculator(x3,n3,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
mu1_cls4,mu2_cls4,sigma1_cls4,sigma2_cls4,lambda1_cls4,lambda2_cls4=ccd_calculator(x4,n4,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)
mu1_cls5,mu2_cls5,sigma1_cls5,sigma2_cls5,lambda1_cls5,lambda2_cls5=ccd_calculator(x5,n5,mu1,sigma1,mu2,sigma2,lambda1,lambda2,mu1_prev,mu2_prev,sigma1_prev,sigma2_prev,lambda1_prev,lambda2_prev)

In [ ]:
sigma1_cls1

In [ ]:
# def class_cond_dens(xi,mu1,mu2,sigma1,sigma2,lambda1,lambda2):
#   value=lambda1*density(mu1,sigma1,xi)+lambda2*density(mu2,sigma2,xi)
#   return value

In [ ]:
uploaded1 = files.upload()

In [ ]:
# Get the uploaded file name
file_name1 = next(iter(uploaded1))

# Load the CSV file using NumPy
data_test = np.genfromtxt(file_name1, delimiter=',')

In [ ]:
X_test=np.zeros((15000,10))
for i in range(0,15000):
  for j in range(0,10):
    X_test[i][j]=data_test[i][j]

In [ ]:
def density_train(x,size,mu1,mu2,sigma1,sigma2,lambda1,lambda2):
  vec=np.zeros((size,1))
  for i in range(0,size):
    vec[i]=class_cond_dens(x[i],mu1,mu2,sigma1,sigma2,lambda1,lambda2)
  return vec


In [ ]:
vec1=density_train(data_train,15000,mu1_cls1,mu2_cls1,sigma1_cls1,sigma2_cls1,lambda1_cls1,lambda2_cls1)
vec2=density_train(data_train,15000,mu1_cls2,mu2_cls2,sigma1_cls2,sigma2_cls2,lambda1_cls2,lambda2_cls2)
vec3=density_train(data_train,15000,mu1_cls3,mu2_cls3,sigma1_cls3,sigma2_cls3,lambda1_cls3,lambda2_cls3)
vec4=density_train(data_train,15000,mu1_cls4,mu2_cls4,sigma1_cls4,sigma2_cls4,lambda1_cls4,lambda2_cls4)
vec5=density_train(data_train,15000,mu1_cls5,mu2_cls5,sigma1_cls5,sigma2_cls5,lambda1_cls5,lambda2_cls5)

In [ ]:
vec5

In [ ]:
post=np.column_stack((vec1,vec2,vec3,vec4,vec5))

In [ ]:
post

In [ ]:
def classifier1(post,p1,p2,p3,p4,p5):
  cls=np.zeros((15000,1))
  a=np.zeros((5,1))
  for i in range(0,15000):
    a[0]=p1*post[i][0]
    a[1]=p2*post[i][1]
    a[2]=p3*post[i][2]
    a[3]=p4*post[i][3]
    a[4]=p5*post[i][4]
    cls[i]=np.argmax(a)+1
    a=np.zeros((5,1))
  return cls

In [ ]:
cls1=classifier1(post,p1,p2,p3,p4,p5)

In [ ]:
y_test=np.zeros((15000,1))
for i in range(0,15000):
  y_test[i]=data_test[i][10]


In [ ]:
def accuracy(arr1,arr2):
  count=0
  for i in range(0,15000):
    if arr1[i]-arr2[i]==0:
      count=count+1
  return count/15000

In [ ]:
accuracy=accuracy(y_test,cls1)

In [ ]:
accuracy